# I. Managing indices and mapppings

Kết nối với ElasticSearch

In [6]:
# 1. initialize the client
from elasticsearch import Elasticsearch

HOST = "https://localhost:9200"
ELASTIC_USER = "elastic"
# The password for the 'elastic' user generated by Elasticsearch
ELASTIC_PASSWORD = "YNhxANqPGqbMGF3ZzH0c"
# The path of ca certificates
CA_CERTS = "C:/Users/ad/elasticsearch-8.6.2/config/certs/http_ca.crt"

# Create the client instance
es = Elasticsearch(
    hosts = HOST,
    ca_certs = CA_CERTS,
    http_auth = (ELASTIC_USER, ELASTIC_PASSWORD),
    verify_certs=False
)
es.info()

C:\Users\ad\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:16: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  
c:\Users\ad\AppData\Local\Programs\Python\Python37-32\lib\site-packages\urllib3\connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


ObjectApiResponse({'name': 'MSI', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'N3fVJWIETLCYlXQF3_ZLkw', 'version': {'number': '8.6.2', 'build_flavor': 'default', 'build_type': 'zip', 'build_hash': '2d58d0f136141f03239816a4e360a8d17b6d8f29', 'build_date': '2023-02-13T09:35:20.314882762Z', 'build_snapshot': False, 'lucene_version': '9.4.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

Tạo index_name và mapping

In [7]:
# 2. create an index
index_name = 'finalproject_index'
type_name = "_search"
if es.indices.exists(index=index_name):
    # delete the index
    es.indices.delete(index=index_name)

# if es.indices.exists(index_name):
#     es.indices.delete(index_name)

es.indices.create(
    index=index_name,
    body={
    "settings": {
    "analysis": {
      "analyzer": {
        "default": {
          "type": "simple"
        },
        "default_search": {
          "type": "simple"
        },
        "my_content_analyzer": {
          "tokenizer": "lowercase",
          "char_filter": [
            "html_strip"
          ],
          "filter": [
            "my_custom_stop_words_filter",
            "my_shingle",
            "trim",
            "remove_duplicates",
            "unique"
          ]
        },
        "my_orthers_analyzer": {
          "tokenizer": "lowercase",
          "char_filter": [
            "html_strip"
          ],
          "filter": [
            "my_shingle",
            "trim",
            "remove_duplicates",
            "unique"
          ]
        }
      },
      "filter": {
        "my_custom_stop_words_filter": {
          "type": "stop",
          "ignore_case": True,
          "stopwords_path": "vietnamese-stopwords.txt"
        },
        "my_shingle": {
          "type": "shingle",
          "max_shingle_size": 2,
          "filler_token": ""
        }
      }
    },
    "number_of_shards": 1,
    "similarity": {
      "my_DFR": {
        "type": "DFR",
        "basic_model": "g",
        "after_effect": "l",
        "normalization": "h2",
        "normalization.h2.c": "3.0"
      },
      "scripted_tfidf": {
        "type": "scripted",
        "weight_script": {
          "source": "double idf = Math.log((field.docCount+1.0)/(term.docFreq+1.0)) + 1.0; return query.boost * idf;"
        },
        "script": {
          "source": "double tf = Math.sqrt(doc.freq); double norm = 1/Math.sqrt(doc.length); return weight * tf * norm;"
        }
      }
    }
  },
  "mappings": {
    "dynamic": False,
    "properties": {
      "Dịch giả": {
        "type": "text",
        "analyzer": "my_orthers_analyzer",
        "search_analyzer": "my_orthers_analyzer",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256,
            "eager_global_ordinals": True
          }
        },
        "store": True,
        "term_vector": "with_positions_offsets"
      },
      "Giá Nhã Nam": {
        "type": "double",
        "coerce": False
      },
      "Giá bìa": {
        "type": "double",
        "coerce": True,
        "store": True
      },
      "Giới thiệu sách": {
        "type": "text",
        "analyzer": "my_content_analyzer",
        "search_analyzer": "my_content_analyzer",
        "term_vector": "with_positions_offsets"
      },
      "Kích thước": {
        "type": "keyword",
        "index": False,
        "eager_global_ordinals": True
      },
      "LinkImage": {
        "type": "keyword",
        "index": False,
        "eager_global_ordinals": True
      },
      "Mã sản phẩm": {
        "type": "keyword",
        "index": True,
        "eager_global_ordinals": True,
        "store": True
      },
      "Ngày phát hành": {
        "type": "date",
        "format": "dd-MM-yyyy",
        "store": True
      },
      "Nhà xuất bản": {
        "type": "text",
        "analyzer": "my_orthers_analyzer",
        "search_analyzer": "my_orthers_analyzer",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256,
            "eager_global_ordinals": True
          }
        },
        "store": True,
        "term_vector": "with_positions_offsets"
      },
      "Danh mục": {
        "type": "text",
        "analyzer": "my_orthers_analyzer",
        "search_analyzer": "my_orthers_analyzer",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256,
            "eager_global_ordinals": True
          }
        },
        "store": True,
        "term_vector": "with_positions_offsets"
      },
      "Số trang": {
        "type": "long",
        "index": False
      },
      "Tác giả": {
        "type": "text",
        "analyzer": "my_orthers_analyzer",
        "search_analyzer": "my_orthers_analyzer",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256,
            "eager_global_ordinals": True
          }
        },
        "store": True,
        "term_vector": "with_positions_offsets"
      },
      "Tên": {
        "type": "text",
        "analyzer": "my_orthers_analyzer",
        "search_analyzer": "my_orthers_analyzer",
        "store": True,
        "term_vector": "with_positions_offsets"
      }
    }
  }
    }
)
es.cluster.health(wait_for_status="yellow")

c:\Users\ad\AppData\Local\Programs\Python\Python37-32\lib\site-packages\urllib3\connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
c:\Users\ad\AppData\Local\Programs\Python\Python37-32\lib\site-packages\urllib3\connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\ad\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:186: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
c:\Users\ad\AppData\Local\Programs\Python\Python37-32\lib\site-packages\urllib3\connectionpo

ObjectApiResponse({'cluster_name': 'elasticsearch', 'status': 'yellow', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 26, 'active_shards': 26, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 9, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 74.28571428571429})

Up dữ liệu lên Elastic

In [8]:
from elasticsearch.helpers import bulk
import json
from datetime import datetime

INPUT_PATH_DATA="dataset/books_dataset_new_2.json"
es.delete_by_query(index=index_name, 
                   query= {
                    "match_all": {}
    })
MyFile= open(INPUT_PATH_DATA,'r').read()
ClearData = MyFile.splitlines(True)
i=0
json_str=""
docs ={}
for line in ClearData:
    source=json.loads(line)
                
    bulk(es, 
    [
        {
        '_index': index_name,
        '_id': i,
        '_source': source
        }
    ])
    i+=1
    
print(len(ClearData))

c:\Users\ad\AppData\Local\Programs\Python\Python37-32\lib\site-packages\urllib3\connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
c:\Users\ad\AppData\Local\Programs\Python\Python37-32\lib\site-packages\urllib3\connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
c:\Users\ad\AppData\Local\Programs\Python\Python37-32\lib\site-packages\urllib3\connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#s

2190


c:\Users\ad\AppData\Local\Programs\Python\Python37-32\lib\site-packages\urllib3\connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
c:\Users\ad\AppData\Local\Programs\Python\Python37-32\lib\site-packages\urllib3\connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
c:\Users\ad\AppData\Local\Programs\Python\Python37-32\lib\site-packages\urllib3\connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#s